In [1]:
import os
import ee
import geemap
import geopandas as gpd
os.chdir("C:/Users/cshat/Projects/Forest-Fires-Europe")

In [2]:
#Run once, token is saved. 
# ee.Authenticate()


Successfully saved authorization token.


In [2]:
ee.Initialize()

In [4]:
# Load your shapefile into a GeoDataFrame
effis_path = "data/processed/Fire_Data/EFFIS/effis-filtered.shp"
effis = gpd.read_file(effis_path)
effis

,Date,COUNTRY,ForestType,AREA_HA,geometry
0,2024-08-21,EL,Conifer,3,"POLYGON ((27.74844 36.14392, 27.74847 36.1434,..."
1,2020-07-16,EL,Mixed,59,"POLYGON ((24.02732 37.69824, 24.02807 37.6975,..."
2,2000-01-01,EL,Mixed,846,"POLYGON ((24.01099 37.72039, 24.013 37.72009, ..."
3,2024-06-29,EL,Mixed,57,"POLYGON ((23.79226 38.18259, 23.79176 38.18186..."
4,2018-10-25,EL,Mixed,610,"POLYGON ((23.94573 40.10388, 23.94572 40.10391..."
...,...,...,...,...,...
12223,2018-05-31,SE,Conifer,58,"POLYGON ((17.92254 64.32946, 17.92275 64.32945..."
12224,2023-06-17,SE,Conifer,25,"POLYGON ((17.6619 64.47057, 17.66232 64.47035,..."
12225,2024-05-30,SE,Conifer,17,"POLYGON ((18.18329 66.05112, 18.18387 66.05074..."
12226,2021-05-30,SE,Conifer,1,"POLYGON ((16.53693 63.50015, 16.53727 63.50019..."


In [7]:
effis['id'] = range(1, len(effis) + 1)

In [8]:
features = []
for _, row in effis.iterrows():
    geom = row['geometry']  # Access the geometry column
    if geom.geom_type == "Polygon":  # Check if it's a single Polygon
        ee_geom = ee.Geometry.Polygon(list(geom.exterior.coords))  # Convert to GEE polygon
    elif geom.geom_type == "MultiPolygon":  # Handle MultiPolygon geometries
        for poly in geom.geoms:
            ee_geom = ee.Geometry.Polygon(list(poly.exterior.coords))  # Convert each part
            feature = ee.Feature(ee_geom, {'id': row['id']})
            features.append(feature)
        continue
    else:
        print(f"Skipping unsupported geometry type: {geom.geom_type}")
        continue  # Skip unsupported geometry types

    feature = ee.Feature(ee_geom, {'id': row['id']})  # Use ID column if available
    features.append(feature)

# Convert to GEE FeatureCollection
feature_collection = ee.FeatureCollection(features)

In [15]:
# Define Landsat image collections
landsat_collections = [
    ee.ImageCollection("LANDSAT/LT05/C02/T1_L2"),
    ee.ImageCollection("LANDSAT/LE07/C02/T1_L2"),
    ee.ImageCollection("LANDSAT/LC08/C02/T1_L2"),
    ee.ImageCollection("LANDSAT/LC09/C02/T1_L2"),
]

# Merge the collections and filter by date range
start_date = "2000-01-01"
end_date = "2025-01-01"
merged_collection = ee.ImageCollection(landsat_collections).filterDate(start_date, end_date)

In [16]:
merged_collection

c:\Users\cshat\Projects\Forest-Fires-Europe\venv\Lib\site-packages\eerepr\repr.py:57: UserWarning: Getting info failed with: 'ImageCollection.fromImages, argument 'images': Invalid type.
Expected type: List<Image<unknown bands>>.
Actual type: List<ImageCollection>.
Actual value: [<ImageCollection>, <ImageCollection>, <ImageCollection>, <ImageCollection>]'. Falling back to string repr.
  warn(f"Getting info failed with: '{e}'. Falling back to string repr.")


In [17]:
# Function to mask clouds and select relevant bands
def mask_and_select(image):
    cloud_mask = image.select("QA_PIXEL").bitwiseAnd(1 << 3).eq(0)
    masked_image = image.updateMask(cloud_mask)
    return masked_image.select(["SR_B4", "SR_B3", "SR_B2", "SR_B5"]).rename(["Red", "Green", "Blue", "NIR"])

# Apply cloud masking and band selection
processed_collection = merged_collection.map(mask_and_select)

In [22]:
# Create an empty list to store results
results = []

# Ensure there is an 'id' column for unique polygon identifiers
if 'id' not in effis.columns:
    effis['id'] = range(1, len(effis) + 1)

In [23]:
for _, row in effis.iterrows():
    # Convert the polygon geometry to an Earth Engine Geometry
    polygon = ee.Geometry.Polygon(row.geometry.__geo_interface__["coordinates"])
    
    # Map over the image collection to calculate median values
    feature_collection = processed_collection.map(
        lambda img: img.reduceRegion(
            reducer=ee.Reducer.median(),
            geometry=polygon,
            scale=30
        ).set({
            "date": img.date().format("YYYY-MM-dd"),
            "polygon_id": row["id"]
        })
    ).filter(ee.Filter.notNull(["Red", "Green", "Blue", "NIR"]))

KeyError: 'geometry'

In [12]:
# Function to calculate median values of bands for polygons
def extract_landsat_time_series(gdf, start_date, end_date):
    # Initialize Earth Engine
    ee.Initialize()

    # Define Landsat image collections (Landsat 5, 7, 8, 9)
    landsat_collections = [
        ee.ImageCollection("LANDSAT/LT05/C02/T1_L2"),
        ee.ImageCollection("LANDSAT/LE07/C02/T1_L2"),
        ee.ImageCollection("LANDSAT/LC08/C02/T1_L2"),
        ee.ImageCollection("LANDSAT/LC09/C02/T1_L2"),
    ]
    
    # Merge the collections and filter by date range
    merged_collection = ee.ImageCollection(landsat_collections).filterDate(start_date, end_date)
    
    # Function to mask clouds and select relevant bands
    def mask_and_select(image):
        cloud_mask = image.select("QA_PIXEL").bitwiseAnd(1 << 3).eq(0)
        masked_image = image.updateMask(cloud_mask)
        return masked_image.select(["SR_B4", "SR_B3", "SR_B2", "SR_B5"]).rename(["Red", "Green", "Blue", "NIR"])

    # Apply masking and band selection to the collection
    processed_collection = merged_collection.map(mask_and_select)

    # Create an empty list to store results
    results = []

    # Loop through each polygon in the GeoDataFrame
    for _, row in gdf.iterrows():
        polygon = ee.Geometry.Polygon(row.geometry.__geo_interface__["geometry"])
        feature_collection = processed_collection.map(
            lambda img: img.reduceRegion(
                reducer=ee.Reducer.median(),
                geometry=polygon,
                scale=30
            ).set({
                "date": img.date().format("YYYY-MM-dd"),
                "polygon_id": row["id"]
            })
        ).filter(ee.Filter.notNull(["Red", "Green", "Blue", "NIR"]))
        
        # Convert FeatureCollection to a pandas DataFrame
        def feature_collection_to_dataframe(fc):
            features = fc.getInfo()["features"]
            data = [
                {**f["properties"], "geometry": row.geometry} for f in features
            ]
            return pd.DataFrame(data)

        fc_df = feature_collection_to_dataframe(feature_collection)
        results.append(fc_df)

    # Concatenate all results into a single GeoDataFrame
    final_df = pd.concat(results, ignore_index=True)
    gdf_results = gpd.GeoDataFrame(final_df, geometry="geometry", crs=gdf.crs)

    return gdf_results

In [13]:
time_series_gdf = extract_landsat_time_series(effis, "2000-01-01", "2025-01-01")

TypeError: Dictionary.set() missing 1 required positional argument: 'value'